# Agents

LLM-powered agents are specialized instances of language models designed to perform specific tasks autonomously. 

Multi-agent collaboration can outperform a single prompt due to several key advantages.

### Complexity Management

LLM-powered agents can break down complex tasks into smaller, manageable sub-tasks. Each agent focuses on a specific part, leading to more efficient and accurate processing.

### Specialization

Different agents can be fine-tuned or designed for particular domains, resulting in expert-level performance in those areas, whereas a single prompt relies on a general-purpose model that may lack depth in specific fields.

### Parallelism

Agents can operate simultaneously on different tasks, significantly speeding up the overall process compared to the sequential nature of handling tasks with a single prompt.

### Scalability

A multi-agent system can easily scale by adding new agents to manage additional tasks, distributing the workload more effectively than trying to scale a single model.

### Redundancy and Error Handling

Multiple agents provide redundancy, enhancing reliability. If one agent fails or performs suboptimally, others can compensate, ensuring the overall task is still completed effectively.

# Setup: packages and environment variables

In [ ]:
import os

import autogen

os.environ["OPENAI_API_KEY"] = ...

# Creating agents

For this purpose, we will use the latest model, GPT-4o.

Less advanced models are less effective in writing code.

In [ ]:
llm_config = {
    "config_list": [{"model": "gpt-4o"}],
}

# User Proxy
UserProxyAgent is a special type of agent that can be used to proxy user input to another agent or group of agents. It supports the following human input modes:
- ALWAYS: Always ask user for input.
- NEVER: Never ask user for input. In this mode, the agent will use the default response (if any) to respond to the message. Or using underlying LLM model to generate response if provided.
- AUTO: Only ask user for input when conversation is terminated by the other agent(s). Otherwise, use the default response (if any) to respond to the message. Or using underlying LLM model to generate response if provided.

We will execute the code natively without docker. This is a security flaw, and in real life it should be avoided, as the execution of arbitrary code can expose sensitive data.

In [ ]:
user_proxy = autogen.UserProxyAgent(
    name="User_proxy",
    system_message="A human admin.",
    code_execution_config={
        "last_n_messages": 3,
        "work_dir": "dataviz_agents",
        "use_docker": False,
    },
    human_input_mode="NEVER",
)

# Coder, Software Engineer, and Data Visualization specialist

Next we define our agents according to their role.

In essence, an agent is definied by its system prompt. Then, it will take part in the conversation as if it was a human being.

We will define:
- a coder, taked with developing the python code to perform a task,
- a software engineer, that will criticise the code written by the coder, and suggest improvements, and
- a data visualization specialist, that should evaluate the quality of the visualization.

Once the chat is triggered, the agents should collaborate autonomously.

In [ ]:
coder = autogen.AssistantAgent(
    name="Coder",
    description="Coder",
    llm_config=llm_config,
)

In [ ]:
sw_eng = autogen.AssistantAgent(
    name="Software_Engineer",
    description="Software Engineer",
    system_message=...,
    llm_config=llm_config,
)

In [ ]:
dataviz_specialist = autogen.AssistantAgent(
    name="Data_Visualization_Specialist",
    description="Data Visualization Specialist",
    system_message=...,
    llm_config=llm_config,
)

In [ ]:
groupchat = autogen.GroupChat(
    agents=[user_proxy, coder, dataviz_specialist, sw_eng], messages=[], max_round=20
)
manager = autogen.GroupChatManager(groupchat=groupchat, llm_config=llm_config)

# Run the chat
Let's trigger the grup chat and see what happens.

In [ ]:
link = "https://raw.githubusercontent.com/uwdata/draco/master/data/cars.csv"
query = "plot a visualization that tells us about the relationship between weight and horsepower"

user_proxy.initiate_chat(
    manager,
    message=f"download data from {link} and use it to plot something that reply to the following query: {query}. "
    f"Save the plot to a file named results.png",
);